In [ ]:
#USEFUL LINKS
# LINK TO REPLACE WORD IN SENTENCE
# https://www.programiz.com/python-programming/methods/string/replace

In [1]:
#all imports
import pickle
import numpy as np
import pandas as pd

import tensorflow_hub as hub

from tensorflow.keras.models import Model

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
import re
from tqdm import tqdm

from nltk.tokenize import word_tokenize 
from nltk import pos_tag 

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Conv2D, Flatten , Input , Conv1D , Concatenate , MaxPooling1D , Dropout , Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
import datetime

from keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from sklearn.metrics import  f1_score , roc_auc_score
import os

import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import nltk.translate.bleu_score as bleu


tf.__version__


'2.4.1'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7E9DjrGpKJUOzvisDP1ySA5p5cVr6EoXNlyFAYFAwltUpxCgHvDZE
Mounted at /content/drive


# DATASET

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Copy of Copy of input_output_LENNN_10.csv')
print(data.shape)
data.head(5)

(202231, 2)


,CORRECT_SENTENCE,ERRONEOUS_SENTENCE
0,Well Prince so Genoa and Lucca are now just fa...,Well Prince so Genoa and Lucca are now just fa...
1,But I warn you if you don t tell me,But I warn you if you don t tell me
2,It was in July 1805 and the speaker was the,It was in July 1805 and the speaker was the
3,With these words she greeted Prince Vasili Kur...,With theze wordz zhe greeted Prince Vazili Kur...
4,Anna Pavlovna had had a cough for some days,Anna Pavlovna had had a cough oor some days


In [4]:
type(data.CORRECT_SENTENCE) , data.isnull().sum().sum()

(pandas.core.series.Series, 0)

In [5]:
length_max = max([len(i) for i in tqdm(data['CORRECT_SENTENCE'])])
length_min = min([len(i) for i in tqdm(data['CORRECT_SENTENCE'])])
avg = [len(i) for i in tqdm(data['CORRECT_SENTENCE'])]
length_avg = np.array([avg]).mean()

length_max , length_min , length_avg

100%|██████████| 202231/202231 [00:00<00:00, 2284972.34it/s]


(94, 1, 44.3706503948455)

In [6]:
CORRECT_SENTENCE_LEN = data['CORRECT_SENTENCE'].str.split().apply(len) 
ERRONEOUS_SENTENCE_LEN = data['ERRONEOUS_SENTENCE'].str.split().apply(len)

In [7]:
for i in range(0,101,10):
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))
for i in range(90,101):
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))
for i in [99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100]:
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))

0 1.0
10 4.0
20 7.0
30 10.0
40 10.0
50 10.0
60 10.0
70 10.0
80 10.0
90 10.0
100 10.0
90 10.0
91 10.0
92 10.0
93 10.0
94 10.0
95 10.0
96 10.0
97 10.0
98 10.0
99 10.0
100 10.0
99.1 10.0
99.2 10.0
99.3 10.0
99.4 10.0
99.5 10.0
99.6 10.0
99.7 10.0
99.8 10.0
99.9 10.0
100 10.0


SINCE 99.9% OF DATA HAS LENGTH LESS THAN 10 , SO SELECTING SENTENCE WITH WORD <10

In [8]:
data['CORRECT_SENTENCE_LEN'] = data['CORRECT_SENTENCE'].str.split().apply(len)
data = data[data['CORRECT_SENTENCE_LEN'] < 20]

data['ERRONEOUS_SENTENCE_LEN'] = data['ERRONEOUS_SENTENCE'].str.split().apply(len)
data = data[data['ERRONEOUS_SENTENCE_LEN'] < 20]

#ADDING start and end IN THE SENTENCES
data['english_inp'] = '<start> ' + data['CORRECT_SENTENCE'].astype(str)
data['english_out'] = data['CORRECT_SENTENCE'].astype(str) + ' <end>'

data = data.drop(['CORRECT_SENTENCE','CORRECT_SENTENCE_LEN','ERRONEOUS_SENTENCE_LEN'], axis=1)
print(data.shape)
data.head()

(202231, 3)


,ERRONEOUS_SENTENCE,english_inp,english_out
0,Well Prince so Genoa and Lucca are now just fa...,<start> Well Prince so Genoa and Lucca are now...,Well Prince so Genoa and Lucca are now just fa...
1,But I warn you if you don t tell me,<start> But I warn you if you don t tell me,But I warn you if you don t tell me <end>
2,It was in July 1805 and the speaker was the,<start> It was in July 1805 and the speaker wa...,It was in July 1805 and the speaker was the <end>
3,With theze wordz zhe greeted Prince Vazili Kur...,<start> With these words she greeted Prince Va...,With these words she greeted Prince Vasili Kur...
4,Anna Pavlovna had had a cough oor some days,<start> Anna Pavlovna had had a cough for some...,Anna Pavlovna had had a cough for some days <end>


In [9]:
data.ERRONEOUS_SENTENCE[3] , data.english_inp[3] , data.english_out[3]

('With theze wordz zhe greeted Prince Vazili Kuragin a man',
 '<start> With these words she greeted Prince Vasili Kuragin a man',
 'With these words she greeted Prince Vasili Kuragin a man <end>')

### Getting train and test 

In [10]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.2)

In [11]:
print(train.shape, validation.shape)
#ADDING TO <end> TO ONE OF THE SENTENCES SO THAT TOKENIZER LEARNS THE WORD <end>
train.iloc[0]['english_inp']= str(train.iloc[0]['english_inp'])+' <end>'
train.iloc[0]['english_out']= str(train.iloc[0]['english_out'])+' <end>'

(161784, 3) (40447, 3)


In [12]:
max_encoder_length = max([len(i) for i in tqdm(train['ERRONEOUS_SENTENCE'])])
max_decoder_length = max([len(i) for i in tqdm(train['english_inp'])])
print(max_encoder_length , max_decoder_length)

100%|██████████| 161784/161784 [00:00<00:00, 1617475.19it/s]

94 102


In [13]:
train.sample(2)

,ERRONEOUS_SENTENCE,english_inp,english_out
48653,The Bree hobbits were in fact friendly and inq...,<start> The Bree hobbits were in fact friendly...,The Bree hobbits were in fact friendly and inq...
96071,He ought to have known,<start> He ought to have known,He ought to have known <end>


In [14]:
validation.sample(2)

,ERRONEOUS_SENTENCE,english_inp,english_out
145414,Proud Mrs,<start> Proud Mrs,Proud Mrs <end>
4256,From privates to general they were not expecti...,<start> From privates to general they were not...,From privates to general they were not expecti...


**TOKENIING**

In [15]:
tknizer_ERRONEOUS_SENTENCE = Tokenizer()
tknizer_ERRONEOUS_SENTENCE.fit_on_texts(train['ERRONEOUS_SENTENCE'].values)
tknizer_CORRECT_SENTENCE = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_CORRECT_SENTENCE.fit_on_texts(train['english_inp'].values)

In [16]:
vocab_size_CORRECT_SENTENCE=len(tknizer_CORRECT_SENTENCE.word_index.keys())
print(vocab_size_CORRECT_SENTENCE)
vocab_size_ERRONEOUS_SENTENCE=len(tknizer_ERRONEOUS_SENTENCE.word_index.keys())
print(vocab_size_ERRONEOUS_SENTENCE)

34512
86363


In [17]:
tknizer_CORRECT_SENTENCE.word_index['<start>'], tknizer_CORRECT_SENTENCE.word_index['<end>']

(1, 21920)

TOKENIZER WITH ENGLISH WORDS

In [18]:
embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_size_CORRECT_SENTENCE+1, 100))
for word, i in tknizer_CORRECT_SENTENCE.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(34513, 100)

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, enc_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder", input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                        = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c


In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
    
    def build(self, input_shape):
        # we are using embedding_matrix weights and not training the embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix],input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
        
    def call(self, target_sentances, state_h, state_c):
        target_embedd           = self.embedding(target_sentances)
        lstm_output, _,_        = self.lstm(target_embedd, initial_state=[state_h, state_c])
        return lstm_output

## Data pipeline

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, max_len):
        self.encoder_inps = data['ERRONEOUS_SENTENCE'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_CORRECT_SENTENCE = tknizer_CORRECT_SENTENCE
        self.tknizer_ERRONEOUS_SENTENCE = tknizer_ERRONEOUS_SENTENCE
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ERRONEOUS_SENTENCE.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)




In [ ]:
class Dataloder(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

## Model

In [ ]:
class vanilla_model(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() 
        self.encoder = Encoder(vocab_size=vocab_size_ERRONEOUS_SENTENCE + 1, embedding_dim=100, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=vocab_size_CORRECT_SENTENCE + 1, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        
        
    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(output, encoder_h, encoder_c)
        output                               = self.dense(decoder_output)
        return output

In [ ]:
train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)

train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
vanilla = vanilla_model(encoder_inputs_length=16,decoder_inputs_length=16,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)
vanilla.compile(optimizer= optimizer, loss= loss_function, metrics=['accuracy'])

In [ ]:
import datetime

# logfile = "/content/Project_2/logs/vanilla/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# #logfile = "C:\\Users\\rpris\\Google Drive\\Project_2\\logs\\vanilla\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tfboard = tf.keras.callbacks.TensorBoard(log_dir=logfile, histogram_freq=1, write_graph=True)

# chkfile = "/content/wts/vanilla/weights-{epoch:02d}-{val_loss:.4f}.hdf5"
# #chkfile2 = "C:\\Users\\rpris\\Google Drive\\Project_2\\wts\\vanilla\\weights-{epoch:02d}-{val_loss:.4f}.hdf5"
# chkpt = tf.keras.callbacks.ModelCheckpoint(chkfile, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=0, mode='min')

# stp = tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss')

In [ ]:
# vanilla.load_weights("/content/drive/MyDrive/Project_2/wts/vanilla/weights-65-0.5898.hdf5")

In [ ]:
# vanilla.summary()

In [ ]:
vanilla  = vanilla_model(encoder_inputs_length=20,decoder_inputs_length=20,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
vanilla.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 50 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
vanilla.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=150 , validation_data=train_dataloader, validation_steps=valid_steps )#, callbacks=[stp, chkpt, tfboard]
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
vanilla.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
157/157 [==============================] - 65s 377ms/step - loss: 3.6994 - val_loss: 3.0375
Epoch 2/150
157/157 [==============================] - 58s 372ms/step - loss: 3.0206 - val_loss: 2.8943
Epoch 3/150
157/157 [==============================] - 60s 383ms/step - loss: 2.8586 - val_loss: 2.7258
Epoch 4/150
157/157 [==============================] - 60s 381ms/step - loss: 2.7114 - val_loss: 2.6221
Epoch 5/150
157/157 [==============================] - 60s 384ms/step - loss: 2.6069 - val_loss: 2.5303
Epoch 6/150
157/157 [==============================] - 60s 381ms/step - loss: 2.5131 - val_loss: 2.4368
Epoch 7/150
157/157 [==============================] - 60s 382ms/step - loss: 2.4173 - val_loss: 2.3289
Epoch 8/150
157/157 [==============================] - 60s 382ms/step - loss: 2.3108 - val_loss: 2.2299
Epoch 9/150
157/157 [==============================] - 60s 382ms/step - loss: 2.2240 - val_loss: 2.1273
Epoch 10/150
157/157 [==============================] - 60s 382m

In [ ]:
os.mkdir('saved_model_newww')

In [ ]:
vanilla.save_weights('saved_model_newww/vanilla.h5')

In [ ]:
vanilla  = vanilla_model(encoder_inputs_length=20,decoder_inputs_length=20,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
vanilla.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 50 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("best_Model.h5",save_best_only=True)
vanilla.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=1 , validation_data=train_dataloader, validation_steps=valid_steps\
                      , callbacks=[model_checkpoint_callback])#, callbacks=[stp, chkpt, tfboard]
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
vanilla.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


113/113 [==============================] - 92s 738ms/step - loss: 4.9768 - val_loss: 4.1294
Model: "vanilla_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_4 (Encoder)          multiple                  10858468  
_________________________________________________________________
decoder_4 (Decoder)          multiple                  3798768   
_________________________________________________________________
dense_4 (Dense)              multiple                  8823067   
Total params: 23,480,303
Trainable params: 23,480,303
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vanilla.load_weights("//content/drive/MyDrive/saved_model_newww/vanilla.h5")

In [ ]:
vanilla.summary()

Model: "vanilla_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Encoder)          multiple                  8990268   
_________________________________________________________________
decoder_1 (Decoder)          multiple                  3822068   
_________________________________________________________________
dense_1 (Dense)              multiple                  8882948   
Total params: 21,695,284
Trainable params: 21,695,284
Non-trainable params: 0
_________________________________________________________________


## Inference

In [ ]:
def inference(enc_inp,dec_inp):
        
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

    #e_input = pad_sequences(e_input, maxlen=16, padding='post')


    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output

    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')

    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            break
    translation += word + " "
    
    return translation
a = train.ERRONEOUS_SENTENCE[4]
b =  train.english_inp[4]
pred = inference(a,b)
pred

'<end> '

In [ ]:
train.ERRONEOUS_SENTENCE[1] , train.english_inp[1] , 

('But I warn you if you don t tell me',
 '<start> But I warn you if you don t tell me')

In [ ]:
enc_inp = train.ERRONEOUS_SENTENCE[1]
dec_inp =  train.english_inp[1] 

In [ ]:
translation=""

e_input=[]
for i in enc_inp.split():
    if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
        e_input.append(0)
    else:
        e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

#e_input = pad_sequences(e_input, maxlen=16, padding='post')


e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

#there is no onestep decoder in this thing, so I have to use the decoder input to predict output

#decoder input
d_input=[]
for i in dec_inp.split():
    if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
        d_input.append(0)
    else:
        d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

#d_input = pad_sequences(d_input, maxlen=16, padding='post')

prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

for word in prediction[0]:
    word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
    if word == "<end>":
        break
    translation += word + " "
translation

'warn warn warn you if you don t tell me '

In [ ]:
sample_train = train.sample(1000)
sample_train

,ERRONEOUS_SENTENCE,english_inp,english_out
14352,Why do you say that replied Princess Mary,<start> Why do you say that replied Princess Mary,Why do you say that replied Princess Mary <end>
66179,The basilisk kills people by looking at them,<start> The basilisk kills people by looking a...,The basilisk kills people by looking at them <...
76953,Several people nearby screamed,<start> Several people nearby screamed,Several people nearby screamed <end>
192440,Madigan J,<start> Madigan J,Madigan J <end>
161989,He wss rhsr rrricer whrm rhe Wsr Minisrry hsd ...,<start> He was that officer whom the War Minis...,He was that officer whom the War Ministry had ...
...,...,...,...
38546,From someohnng ohao he oold me nn our journey ...,<start> From something that he told me in our ...,From something that he told me in our journey ...
189090,He encouraged me Davison,<start> He encouraged me Davison,He encouraged me Davison <end>
13207,Only for Christ s sake sake,<start> Only for Christ s sake,Only for Christ s sake <end>
57492,He left without another word,<start> He left without another word,He left without another word <end>


In [ ]:
def inference(enc_inp,dec_inp):
            
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

    #e_input = pad_sequences(e_input, maxlen=16, padding='post')


    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output

    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')

    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            break
        translation += word + " "
    return translation
a = train.ERRONEOUS_SENTENCE[5]
b =  train.english_inp[5]
pred = inference(a,b)
pred

'was was as she said suffering from la grippe grippe '

In [ ]:
a = train.ERRONEOUS_SENTENCE[10]
b =  train.english_inp[10]
pred = inference(a,b)
a  , b , pred

('He spoke in that refined French in which our grandfathers',
 '<start> He spoke in that refined French in which our grandfathers',
 'spoke spoke in that refined in in which came regiment ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[12]
b =  train.english_inp[12]
pred = inference(a,b)
a  , b , pred

('First of all dear friend tell me how you are',
 '<start> First of all dear friend tell me how you are',
 'of of all you friend tell me how you are ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[200]
b =  train.english_inp[200]
pred = inference(a,b)
a  , b , pred

('He spoke with such self confixence that his hearers coulx',
 '<start> He spoke with such self confidence that his hearers could',
 'spoke spoke with such self confidence that the hearers thought ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[300]
b =  train.english_inp[300]
pred = inference(a,b)
a  , b , pred

('The sovereigns Why y are sending ambassadors to compliment usurper',
 '<start> The sovereigns Why they are sending ambassadors to compliment the',
 'negroes negroes they they are sending to to compliment ease honourable ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[500]
b =  train.english_inp[500]
pred = inference(a,b)
a  , b , pred

('What is it you are afraid of Lise I don',
 '<start> What is it you are afraid of Lise I don',
 'is is it you are afraid of don don don ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[1000]
b =  train.english_inp[1000]
pred = inference(a,b)
a  , b , pred

('Secrets indeed All have secrets ff their fwn answered Natasha',
 '<start> Secrets indeed All have secrets of their own answered Natasha',
 'indeed indeed you you your of their own answered ')

In [ ]:
sample_train = train.sample(1000)
result = []
i=0
for i, j , k in tqdm(sample_train.values):
    # print(i , j , k)
    pred = inference(i , j)
    result.append(pred)

result


100%|██████████| 1000/1000 [00:52<00:00, 18.87it/s]


['on on the following morning every hope of this kind ',
 'any any case i i going to call you by ',
 'that that that that the sand sand back on the ',
 'came and taking my arm led me to a building ',
 'has has has never a friend friend nor do ',
 'forcheville forcheville introduced on by his had had ceased to ',
 'kuragin kuragin said said s s having a laugh ',
 'four four date said said lifting lifting his head ',
 'she wasn t exactly hard to miss ',
 'dear dear s a natural ',
 'r ',
 'sir sir is how much this opinion is life furthering ',
 'paulina paulina mayor mayor to to ',
 'had had one his father s hooked nose ',
 'many many organizations wanted to obtain dinners in his honor ',
 'was was not ',
 'forgive forgive me that that that ',
 'he he yelled pointing at a a head ',
 'you you re all in my means means should hand ',
 'the the upper part of the the masses of concrete ',
 'my greatgreat grandfather see charmingly charmingly headmaster ever ever had ',
 'see see said the old m

In [ ]:
sample_train['PREDICTED_SENTENCE']= result
sample_train.head(20)

,ERRONEOUS_SENTENCE,english_inp,english_out,PREDICTED_SENTENCE
37624,But on the following morning every hope of thi...,<start> But on the following morning every hop...,But on the following morning every hope of thi...,on on the following morning every hope of this...
129898,In any case I m going to call you by,<start> In any case I m going to call you by,In any case I m going to call you by <end>,any any case i i going to call you by
165511,The treth is that the Senator is back on the,<start> The truth is that the Senator is back ...,The truth is that the Senator is back on the <...,that that that that the sand sand back on the
10084,came and oaking my arm led me oo a building,<start> came and taking my arm led me to a bui...,came and taking my arm led me to a building <end>,came and taking my arm led me to a building
108058,Lord Voldemort has never a friend nor do I bel...,<start> Lord Voldemort has never had a friend ...,Lord Voldemort has never had a friend nor do I...,has has has never a friend friend nor do
193868,His actors spurred on by him had had not ceased,<start> His actors spurred on by him had not c...,His actors spurred on by him had not ceased to...,forcheville forcheville introduced on by his h...
121821,Behind him Ron Ron said He s having a laugh,<start> Behind him Ron said He s having a laugh,Behind him Ron said He s having a laugh <end>,kuragin kuragin said said s s having a laugh
80480,Really romantic date Hagrid said Charlie shaki...,<start> Really romantic date Hagrid said Charl...,Really romantic date Hagrid said Charlie shaki...,four four date said said lifting lifting his h...
80442,she wasn t exactly hard to miss,<start> she wasn t exactly hard to miss,she wasn t exactly hard to miss <end>,she wasn t exactly hard to miss
58488,The boy s a a natural,<start> The boy s a natural,The boy s a natural <end>,dear dear s a natural


## BLEU Score

In [ ]:
import nltk

In [ ]:
inp = sample_train.ERRONEOUS_SENTENCE.values[20]
translate = sample_train.PREDICTED_SENTENCE.values[20]
print(inp)
print(translate)
print('BLEU score: {}'.format(nltk.translate.bleu_score.sentence_bleu(inp, translate)))

my greatgreat grandfather see Least vovular headmaster Hogwarts ever had
my greatgreat grandfather see charmingly charmingly headmaster ever ever had 
BLEU score: 0.6643548861507491


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
import nltk.translate.bleu_score as bleu

score = 0
for i in tqdm(range(1000)):
    inp = sample_train.ERRONEOUS_SENTENCE.values[i]

    # print(inp)
    translate = sample_train.PREDICTED_SENTENCE.values[i]
    # print(translate)
    bleu = nltk.translate.bleu_score.sentence_bleu(inp, translate)
    score = score + bleu
average_bleu = score / 1000
print('the average BLEU score of these sentences. are :' , average_bleu)



  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


  3%|▎         | 34/1000 [00:00<00:02, 334.27it/s]

  6%|▋         | 65/1000 [00:00<00:02, 326.06it/s]

 10%|▉         | 98/1000 [00:00<00:02, 326.53it/s]

 13%|█▎        | 131/1000 [00:00<00:02, 322.14it/s]

 16%|█▌        | 161/1000 [00:00<00:02, 313.26it/s]

 19%|█▉        | 190/1000 [00:00<00:02, 305.89it/s]

 22%|██▏       | 220/1000 [00:00<00:02, 303.28it/s]

 25%|██▍       | 248/1000 [00:00<00:02, 290.23it/s]

 28%|██▊       | 277/1000 [00:00<00:02, 288.34it/s]

 31%|███       | 309/1000 [00:01<00:02, 296.62it/s]

 34%|███▍      | 339/1000 [00:01<00:02, 296.12it/s]

 37%|███▋      | 369/1000 [00:01<00:02, 283.15it/s]

 40%|████      | 400/1000 [00:01<00:02, 288.41it/s]

 44%|████▍     | 438/1000 [00:01<00:01, 308.56it

the average BLEU score of these sentences. are : 0.7350925622799785


# ATTENTION

In [19]:
#all imports
import pickle
import numpy as np
import pandas as pd

import tensorflow_hub as hub
from tensorflow.keras.models import Model

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
import re
from tqdm import tqdm

from nltk.tokenize import word_tokenize 
from nltk import pos_tag 

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Conv2D, Flatten , Input , Conv1D , Concatenate , MaxPooling1D , Dropout , Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
import datetime

from keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from sklearn.metrics import  f1_score , roc_auc_score
import os

import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import nltk.translate.bleu_score as bleu


tf.__version__


'2.4.1'

### <font color='blue'>**Implement custom encoder decoder and attention layers**</font>

<font color='blue'>**Encoder**</font>

In [20]:
class Encoder(tf.keras.Model):
    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        super().__init__()
        self.inp_vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
        self.embedding = Embedding(input_dim=self.inp_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):
        input_embedd = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd, states)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    
    def initialize_states(self,batch_size):
      self.lstm_state_h = tf.zeros([batch_size , self.lstm_size])
      self.lstm_state_c = tf.zeros([batch_size , self.lstm_size])
      return self.lstm_state_h,self.lstm_state_c


<font color='blue'>**Attention**</font>

In [21]:
class Attention(tf.keras.layers.Layer):

  def __init__(self,scoring_function, att_units):
    super(Attention, self).__init__()
    self.scoring_function = scoring_function
    self.att_units = att_units
    self.W1 = tf.keras.layers.Dense(att_units)
    self.W2 = tf.keras.layers.Dense(att_units)
    self.V = tf.keras.layers.Dense(1)

  
  def call(self,decoder_hidden_state,encoder_output):
    
    if self.scoring_function == 'dot':
        decoder_hidden_state_reshaped = tf.reshape(decoder_hidden_state , (decoder_hidden_state.shape[0],decoder_hidden_state.shape[1],1))

        #I WAS USING tf.keras.layers.Dot FOR DOT PRODUCT , BUT IT GAVE INCOMPATIBILITY IN SHAPES , SO NOW I VE USED tf.keras.layers.dot
        score =  tf.keras.layers.dot([ encoder_output , decoder_hidden_state_reshaped] , [2,1]) 
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    elif self.scoring_function == 'general':
        decoder_hidden_state_reshaped = tf.reshape(decoder_hidden_state , (decoder_hidden_state.shape[0],decoder_hidden_state.shape[1],1))
        W = tf.random.uniform(shape=[encoder_output.shape[0] , self.att_units , self.att_units])
        score =  tf.keras.layers.dot([ encoder_output , W] , [2,1]) 
        score =  tf.keras.layers.dot([ score , decoder_hidden_state_reshaped] , [2,1]) 
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    elif self.scoring_function == 'concat':

        decoder_hidden_state_reshaped = tf.expand_dims(decoder_hidden_state, 1)
        score =  self.V(tf.nn.tanh(self.W1(decoder_hidden_state_reshaped) + self.W2(encoder_output)) )
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    
    

<font color='blue'>**OneStepDecoder**</font>

In [22]:
class One_Step_Decoder(tf.keras.Model):
    def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(One_Step_Decoder, self).__init__()

        # Initialize decoder embedding layer, LSTM 
        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

        self.attention=Attention(score_fun,att_units)
        self.embedding = tf.keras.layers.Embedding(tar_vocab_size, embedding_dim)
        self.lstm = LSTM(self.dec_units , return_state=True, return_sequences=True, name="Decoder_LSTM")
        self.dense = tf.keras.layers.Dense(self.tar_vocab_size)

    def call(self,input_to_decoder, encoder_output, state_h,state_c):
        output = self.embedding(input_to_decoder) # (32, 1, 12)
        context_vector,attention_weights=self.attention(state_h,encoder_output)
        concat = tf.concat([tf.expand_dims(context_vector, 1), output], axis=-1)
        lstm_output, state_h, state_c = self.lstm(concat)
        
        output = self.dense(lstm_output)
        output = tf.reshape(output, (-1, output.shape[2]))

        return output,state_h,state_c,attention_weights,context_vector


<font color='blue'>**Decoder**</font>

In [23]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(Decoder , self).__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

        self.onestep_decoder=One_Step_Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)

    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        all_outputs = tf.TensorArray(tf.float32, size=tf.shape(input_to_decoder)[1])
        for timestep in range(tf.shape(input_to_decoder)[1]):
            output,state_h,state_c,attention_weights,context_vector = self.onestep_decoder(input_to_decoder[: , timestep : timestep + 1] , \
                                                                                           encoder_output , decoder_hidden_state , decoder_cell_state)


            all_outputs = all_outputs.write(timestep , output)
        all_outputs = tf.transpose(all_outputs.stack() , [1,0,2])
        return all_outputs




In [24]:
out_vocab_size=13 
embedding_dim=12 
input_length=10
dec_units=16 
att_units=16
batch_size=32

target_sentences=tf.random.uniform(shape=(batch_size,input_length),maxval=10,minval=0,dtype=tf.int32)
encoder_output=tf.random.uniform(shape=[batch_size,input_length,dec_units])
state_h=tf.random.uniform(shape=[batch_size,dec_units])
state_c=tf.random.uniform(shape=[batch_size,dec_units])
score_fun = 'concat'
decoder=Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)
output=decoder(target_sentences,encoder_output, state_h, state_c)
print(output.shape)

(32, 10, 13)


<font color='blue'>**Encoder Decoder model**</font>

In [25]:
class Dataset:
    def __init__(self, data, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, max_len):
        self.encoder_inps = data['ERRONEOUS_SENTENCE'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_CORRECT_SENTENCE = tknizer_CORRECT_SENTENCE
        self.tknizer_ERRONEOUS_SENTENCE = tknizer_ERRONEOUS_SENTENCE
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ERRONEOUS_SENTENCE.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)


class Dataloder(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [26]:
train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)

train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [27]:
class encoder_decoder(tf.keras.Model):
    def __init__(self,score_fun , encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__()
        #encoder decoder

        self.score_fun = score_fun

        self.encoder=Encoder(inp_vocab_size = vocab_size_ERRONEOUS_SENTENCE+1,embedding_size = 50,lstm_size = 64,input_length = encoder_inputs_length)
        self.decoder=Decoder(out_vocab_size = vocab_size_CORRECT_SENTENCE+1, embedding_dim = 100, input_length = decoder_inputs_length, dec_units =  64 \
                             ,score_fun = self.score_fun ,att_units = 64)

    def call(self,data):

        input,output = data[0], data[1]
        initial_state= self.encoder.initialize_states(batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input , initial_state)

        decoder_output= self.decoder(output,encoder_output, encoder_h, encoder_c)

        return decoder_output


In [29]:
batch_size=512
score_fun  = 'general'
att_units = 64
model_2  = encoder_decoder(score_fun = score_fun , encoder_inputs_length=20,decoder_inputs_length=10,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
model_2.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 20 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
log_dir="logs1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model_2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=150, validation_data=train_dataloader, validation_steps=valid_steps , callbacks = checkpoint)
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
model_2.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
157/157 [==============================] - 88s 532ms/step - loss: 5.3506 - val_loss: 4.1200
Epoch 2/150
157/157 [==============================] - 81s 519ms/step - loss: 4.0746 - val_loss: 3.9506
Epoch 3/150
157/157 [==============================] - 81s 519ms/step - loss: 3.8922 - val_loss: 3.9636
Epoch 4/150
157/157 [==============================] - 82s 519ms/step - loss: 3.8116 - val_loss: 3.9484
Epoch 5/150
157/157 [==============================] - 82s 519ms/step - loss: 3.9153 - val_loss: 3.8861
Epoch 6/150
157/157 [==============================] - 81s 518ms/step - loss: 3.7552 - val_loss: 3.8725
Epoch 7/150
157/157 [==============================] - 81s 516ms/step - loss: 3.7337 - val_loss: 3.8188
Epoch 8/150
157/157 [==============================] - 81s 517ms/step - loss: 3.7929 - val_loss: 3.8618
Epoch 9/150
157/157 [==============================] - 81s 517ms/step - loss: 3.7123 - val_loss: 3.8828
Epoch 10/150
157/157 [==============================] - 81s 518m

In [30]:
os.mkdir('saved_model_attention')

In [33]:
model_2.save_weights('saved_model_attention/attention.h5')

In [36]:
model_2.load_weights("/content/drive/MyDrive/saved_model_attention/attention.h5")
model_2.summary()

Model: "encoder_decoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Encoder)          multiple                  4347640   
_________________________________________________________________
decoder_2 (Decoder)          multiple                  5753269   
Total params: 10,100,909
Trainable params: 10,100,909
Non-trainable params: 0
_________________________________________________________________


## <font color='blue'>**Inference**</font>

<font color='blue'>**CORRECTING THE INCORRECT SENTENCES**</font>

In [37]:
def predict(input_sentence):
    pred = []
    input_sequence = tknizer_CORRECT_SENTENCE.texts_to_sequences([input_sentence])
    result = ' '
    encoder_seq = pad_sequences(input_sequence, maxlen=20, dtype='int32', padding='post')  
    initial_state = model_2.layers[0].initialize_states(1)
    encoder_output, encoder_h, encoder_c = model_2.layers[0](tf.constant(encoder_seq), initial_state)
    start_index = tf.constant([[tknizer_CORRECT_SENTENCE.word_index['<start>']]])
    states = [encoder_h, encoder_c]

    for i in range(20): 
        # print(start_index)
        predicted_out,state_h,state_c,attention_weights,context_vector = model_2.layers[1].onestep_decoder(start_index, encoder_output , encoder_h, encoder_c )
        attention_weights = tf.reshape(attention_weights, (-1, ))
        start_index = np.reshape(np.argmax(predicted_out), (1, 1))
        pred.append(tknizer_CORRECT_SENTENCE.index_word[start_index[0][0]])
        if(pred[-1]=='<end>'):
            break
        translated_sentence = ' '.join(pred)
    return translated_sentence

inp = validation.values[2000][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

Figg had perched hersela nervously on the very edge oa
and
BLEU score: 1.0


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [38]:
inp = validation.values[2][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

He s just started at the Ministry and this is
i have the
BLEU score: 0.8801117367933934


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [39]:
inp = validation.values[20][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

Io he were his genius oor light music would be
i have the
BLEU score: 0.8408964152537145


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [40]:
inp = validation.values[2000][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

Figg had perched hersela nervously on the very edge oa
and have the
BLEU score: 0.9036020036098448


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [41]:
#USING THIS METHOD TO GET THE BLEU SCORE , THE REFERENCE NOTEBOOK METHOD SHOWS ME AN ERROR FOR 1000 FILES , BUT IT WORKS FINE FOR SINGLE FILE
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [42]:
inp = validation.values[2][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(nltk.translate.bleu_score.sentence_bleu(inp, translate)))

He s just started at the Ministry and this is
i have the
BLEU score: 0.8801117367933934


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [43]:
sample_train = train.sample(1000)
result = []
i=0
for i, j , k in tqdm(sample_train.values):
    # print(i , j , k)
    pred = predict(i)
    result.append(pred)

result

100%|██████████| 1000/1000 [00:46<00:00, 21.37it/s]


['i have been to have been to have been to have been to have been to have been to have',
 'asked',
 'i have the',
 'they have the',
 'you don t have the',
 'and have the',
 'i have the',
 'he asked',
 'i have to have to have to have to have to have to have to have to have to have',
 'harry could have the',
 'he asked',
 'he asked',
 'he asked',
 'he asked',
 'i have the',
 'i have the',
 'i have the',
 'when they have been to be asked',
 'he s going to an',
 'he was a',
 'have the',
 'and have the',
 'his',
 'he no more he no more he no more he no more he no more he no more he no',
 'have the',
 'and have come and have come and have been to have been to have the',
 'have the',
 'the',
 'he asked',
 'she have the',
 'i have the',
 'he',
 'he',
 'when i have the',
 'this have the',
 'i have an',
 'from him from him from him from him from him from him from him from him from him from him',
 'as if the',
 'i have been to have been to have been to have been to have been to have the',
 'he',


In [44]:
sample_train['PREDICTED_SENTENCE']= result
sample_train.head(20)

,ERRONEOUS_SENTENCE,english_inp,english_out,PREDICTED_SENTENCE
197645,That makes no no difference to me said Jehan tis,<start> That makes no difference to me said Je...,That makes no difference to me said Jehan tis ...,i have been to have been to have been to have ...
99162,Couldn t get past the gargoyle,<start> Couldn t get past the gargoyle,Couldn t get past the gargoyle <end>,asked
107285,That s me outsmarted nnnnt All rngh come nn then,<start> That s me outsmarted innit All righ co...,That s me outsmarted innit All righ come in th...,i have the
28930,LADY ANNE Would they were basilisks to strike ...,<start> LADY ANNE Would they were basilisks to...,LADY ANNE Would they were basilisks to strike ...,they have the
67247,The most they could do however was to lock away,<start> The most they could do however was to ...,The most they could do however was to lock awa...,you don t have the
134317,The curse of God on thee for a blockhead sakd,<start> The curse of God on thee for a blockhe...,The curse of God on thee for a blockhead said ...,and have the
87438,Then he was gone,<start> Then he was gone,Then he was gone <end>,i have the
26381,Resofrnng ohe essenonal cfndnonfn ff relaonfn ...,<start> Restoring the essential condition of r...,Restoring the essential condition of relation ...,he asked
202068,He clung to it with desjerate hands and at the,<start> He clung to it with desperate hands an...,He clung to it with desperate hands and at the...,i have to have to have to have to have to have...
157665,But But this period was important for another ...,<start> But this period was important for anot...,But this period was important for another reas...,harry could have the


<font color='blue'>**Calculate BLEU score**</font>

In [45]:
#Create an object of your custom model.
#Compile and train your model on dot scoring function.
# Visualize few sentences randomly in Test data
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

#Sample example
import nltk.translate.bleu_score as bleu
reference = ['i am groot'.split(),] # the original
translation = 'it is ship'.split() # trasilated using model
print('BLEU score: {}'.format(bleu.sentence_bleu(reference, translation)))

BLEU score: 0


In [46]:
import nltk.translate.bleu_score as bleu

score = 0
for i in tqdm(range(1000)):
    inp = sample_train.ERRONEOUS_SENTENCE.values[i]

    # print(inp)
    translate = sample_train.PREDICTED_SENTENCE.values[i]
    # print(translate)
    bleu = nltk.translate.bleu_score.sentence_bleu(inp, translate)
    score = score + bleu
average_bleu = score / 1000
print('the average BLEU score of these sentences. are :' , average_bleu)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
100%|██████████| 1000/1000 [00:01<00:00, 826.61it/s]

the average BLEU score of these sentences. are : 0.822307632753043
